In [224]:
import pandas as pd
import numpy as np
import random
from dotenv import load_dotenv
import os
import openai
from github import Github

load_dotenv()
key = os.getenv("GITHUB_API_KEY")
g = Github(key)

### 1. Get Commits and Related Categories

In [225]:

df = pd.read_excel("./data/RQ1.xlsx")
df = df.drop(columns=["Unnamed: 2", "Count", "Count.1", "Unnamed: 5", "Unnamed: 8", "Count-subcatagory", "Count-Sub-Sub-Sub-Category", "Unnamed: 11", "Solution Category", "Tag", "Sub-Sub-Category","Sub-Sub-Sub-Category", "Sub-Sub-Sub-sub-Category"])

df

,Category,Sub-Category,Solution,Solution Main Category,Link to commit
0,Inefficient coding for target micro-architecure,Memory/Data locality,Using std:array instead of std:Set,Data structure optimization for data locality,https://github.com/CGAL/cgal/commit/28a9cb150a...
1,NaN,NaN,NaN,Data structure optimization for data locality,https://github.com/CGAL/cgal/commit/b0379f54bd...
2,NaN,NaN,regular memory access,Data structure optimization for data locality,https://github.com/mlpack/mlpack/commit/723dea...
3,NaN,NaN,Using std:vector instead of std:List,Data structure optimization for data locality,https://github.com/CGAL/cgal/commit/8855eb54c3...
4,NaN,NaN,NaN,Avoid memory dereference by storing in register,https://github.com/lammps/lammps/commit/6c5edf...
...,...,...,...,...,...
182,Unintentional Programming logic error,boundary condition check,NaN,fix programming error,https://github.com/xianyi/OpenBLAS/commit/274d...
183,NaN,NaN,NaN,fix programming error,https://github.com/libMesh/libmesh/commit/35e6...
184,NaN,NaN,NaN,fix programming error,https://github.com/mlpack/mlpack/commit/205b05...
185,Inefficiency due to new compiler version,NaN,NaN,Reduce parallelization,https://github.com/lattice/quda/commit/2ae74fd...


In [226]:
prev_category = "Inefficient coding for target micro-architecure"
prev_sub_category = "Memory/Data locality"

for row in df.iterrows():
    category = row[1]["Category"]
    if (pd.isnull(category)):
        row[1]["Category"] = prev_category
    else:
        prev_category = row[1]["Category"]
    
    sub_category = row[1]["Sub-Category"]
    if (pd.isnull(sub_category)):
        row[1]["Sub-Category"] = prev_sub_category
    else:
        prev_sub_category = row[1]["Sub-Category"]

df


,Category,Sub-Category,Solution,Solution Main Category,Link to commit
0,Inefficient coding for target micro-architecure,Memory/Data locality,Using std:array instead of std:Set,Data structure optimization for data locality,https://github.com/CGAL/cgal/commit/28a9cb150a...
1,Inefficient coding for target micro-architecure,Memory/Data locality,NaN,Data structure optimization for data locality,https://github.com/CGAL/cgal/commit/b0379f54bd...
2,Inefficient coding for target micro-architecure,Memory/Data locality,regular memory access,Data structure optimization for data locality,https://github.com/mlpack/mlpack/commit/723dea...
3,Inefficient coding for target micro-architecure,Memory/Data locality,Using std:vector instead of std:List,Data structure optimization for data locality,https://github.com/CGAL/cgal/commit/8855eb54c3...
4,Inefficient coding for target micro-architecure,Memory/Data locality,NaN,Avoid memory dereference by storing in register,https://github.com/lammps/lammps/commit/6c5edf...
...,...,...,...,...,...
182,Unintentional Programming logic error,boundary condition check,NaN,fix programming error,https://github.com/xianyi/OpenBLAS/commit/274d...
183,Unintentional Programming logic error,boundary condition check,NaN,fix programming error,https://github.com/libMesh/libmesh/commit/35e6...
184,Unintentional Programming logic error,boundary condition check,NaN,fix programming error,https://github.com/mlpack/mlpack/commit/205b05...
185,Inefficiency due to new compiler version,boundary condition check,NaN,Reduce parallelization,https://github.com/lattice/quda/commit/2ae74fd...


In [307]:
category_name_to_num = {
    "Inefficient coding for target micro-architecure": 1,
    "Missing parallelism": 2,
    "Parallelization overhead/inefficiency": 3,
    "Inefficient Concurrency control and synchronization": 4,
    "Unnecessary process communication": 5,
    "Inefficient algorithm /data-structure and their implementation": 6,
    "Inefficient memory management": 7,
    "I/O inefficiency": 8,
    "Unintentional Programming logic error": 9,
    "Inefficiency due to new compiler version": 10
}

sub_category_name_to_num = {
    "Memory/Data locality": 1.1,
    "Micro-architectural inefficiency": 1.2,
    "Vector/SIMD parallelism": 2.1,
    "GPU parallelism": 2.2,
    "Instruction level parallelism": 2.3,
    "Task parallelism": 2.4,
    "Small parallel region": 3.1,
    "Inefficeint thread mapping / inefficient block size / Load imbalance": 3.2,
    "Under-parallelization": 3.3,
    "Over-Parallelization": 3.4,
    "Unncessary locks": 4.1,
    "Unnecessary strong memory consistency": 4.2,
    "Lock management overhead": 4.3,
    "Unnecessary synchronization": 4.4,
    "Unnecessary operation/traversal/function call": 6.1,
    "Redundant operation": 6.2,
    "Expensive operation": 6.3,
    "Frequent  function call": 6.4,
    "Inefficient data-structure library": 6.5,
    "Usage of improper data type": 6.6,
    "memory leak": 7.1,
    "repreated memory allocation": 7.2,
    "Redundant memory allocation": 7.3,
    "Slower memory allocation library call": 7.4,
    "Insufficient memory": 7.5,
    "unnecessary data copy": 7.6,
    "Sequential I/O operation": 8.1,
    "Over parallelization": 8.2
}

QUESTION = "Following code has some performance related bugs.\nPlease identify the performance bugs category and subcategory from the given list (pick only one):\n\n"

LIST = "LIST:\n"

LIST += "1) Inefficient coding for target micro-architecture\n"
LIST += "   1.1) Memory/Data locality\n"
LIST += "   1.2) Micro-architectural inefficiency\n"
	
LIST += "2)	Missing parallelism\n"
LIST += "   2.1) Vector/SIMD parallelism\n"
LIST += "   2.2) GPU parallelism\n"
LIST += "   2.3) Instruction level parallelism\n"
LIST += "   2.4) Task parallelism\n"

LIST += "3)	Parallelization overhead/inefficiency\n"
LIST += "   3.1) Small parallel region\n"
LIST += "   3.2) Inefficient thread mapping / inefficient block size / Load imbalance\n"
LIST += "   3.3) Under-parallelization\n"
LIST += "   3.4) Over-Parallelization\n"

LIST += "4)	Inefficient Concurrency control and synchronization\n"
LIST += "   4.1) Unnecessary locks\n"
LIST += "   4.2) Unnecessary strong memory consistency\n"
LIST += "   4.3) Lock management overhead\n"
LIST += "   4.4) Unnecessary synchronization\n"
	
LIST += "5)	Unnecessary process communication\n"

LIST += "6)	Inefficient algorithm /data-structure and their implementation\n"
LIST += "   6.1) Unnecessary operation/traversal/function call\n"
LIST += "   6.2) Redundant operation\n"
LIST += "   6.3) Expensive operation\n"
LIST += "   6.4) Frequent function call\n"
LIST += "   6.5) Inefficient data-structure library\n"
LIST += "   6.6) Usage of improper data type\n"
	
LIST += "7)	Inefficient memory management\n"
LIST += "   7.1) Memory leak\n"
LIST += "   7.2) Repeated memory allocation\n"
LIST += "   7.3) Redundant memory allocation\n"
LIST += "   7.4) Slower memory allocation library call\n"
LIST += "   7.5) Insufficient memory\n"
LIST += "   7.6) unnecessary data copy\n"

LIST += "8)	I/O inefficiency\n"
LIST += "   8.1) Sequential I/O operation\n"
LIST += "   8.2) Over parallelization\n"

LIST += "9)	Unintentional Programming logic error\n"

LIST += "10) Inefficiency due to new compiler version\n"

### 2. Automatic Prompt Generation

In [264]:

def get_source_code(link):
    # Last 40 characters of the link is commit sha.
    sha = link[-40:]

    # Repository organization and name:
    repo = link[19:-48]

    # g is the github object created by API key.
    repo = g.get_repo(repo)
    commit = repo.get_commit(sha=sha)
    old_sha = commit.parents[0].sha
    path = commit.files[0].filename
    old_contents = repo.get_contents(path=path, ref=old_sha)
    old_code = old_contents.decoded_content.decode("utf-8")
    return old_code

  
def generate_prompt(link):
    # Automatic prompt generator from a github commit link.
    prompt = QUESTION
    prompt += LIST
    prompt += "\n"
    prompt += "FILE:\n"
    code = get_source_code(link)
    prompt += code
    prompt += "\n\n"

    # Write the prompt into a file.
    fname = "./data/prompt.txt"
    with open(fname, "w") as file:
        file.write(prompt)


def print_row(i):
    row = df.iloc[i]
    category = row["Category"]
    sub_category = row["Sub-Category"]
    link = row["Link to commit"]

    print(f"Row: {i}")
    print(f"Category: {category_name_to_num[category]}) {category}")
    print(f"Sub-Category: {sub_category_name_to_num[sub_category]}) {sub_category}")
    print(f"Link: {link}")

In [229]:
index_list = list(range(0, 187))
shuffled_index_list = []

while len(index_list) > 0:
    i = random.choice(index_list)
    index_list.remove(i)
    shuffled_index_list.append(i)

In [321]:
IDX = 57
current = shuffled_index_list[IDX]
link = df.iloc[IDX]["Link to commit"]
print_row(current)
prompt = generate_prompt(link)

Row: 78
Category: 3) Parallelization overhead/inefficiency
Sub-Category: 3.2) Inefficeint thread mapping / inefficient block size / Load imbalance
Link: https://github.com/gromacs/gromacs/commit/cd01238b6b0eca4ddf115efc3abda44e98eabe6d


### 2. ChatGPT Through API

In [ ]:
load_dotenv()
key = os.getenv("OPENAI_API_KEY")
openai.api_key = key

In [ ]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    temperature=0.2,
    max_tokens=1000,
    messages=[{"role": "user", "content": "Hey there"}]
)

response.choices[0].message